In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
from poke_env.environment.weather import *
from StatCalculator import *
from Data.TypeChart import *

In [10]:
# Returns amount of damage as a ratio of lost hp to target's full hp
def damage_calc_approx(user, userDex, target, targetDex, level, move, weather):
    power = move['basePower']
    
    atkBase = userDex['baseStats']['atk'] if move['category'] == 'Physical' else userDex['baseStats']['spa']
    atkEv = user['evs']['atk'] if move['category'] == 'Physical' else user['evs']['spa']
    atkIv = user['ivs']['atk'] if move['category'] == 'Physical' else user['ivs']['spa']
    atk = non_hp_stat_calc(atkBase, atkEv, atkIv, level, atkNatureVal)
    
    defBase = targetDex['baseStats']['def'] if move['category'] == 'Physical' else targetDex['baseStats']['spd']
    defEv = target['evs']['def'] if move['category'] == 'Physical' else target['evs']['spd']
    defIv = target['ivs']['def'] if move['category'] == 'Physical' else target['ivs']['spd']
    dfn = non_hp_stat_calc(defBase, defEv, defIv, level, defNatureVal)
    
    weatherEff = weather_effect(weather, move['type'])
    stab = 1.5 if move['type'] in user['types'] else 1.0
    typeEff = get_type_weakness(targetDex['types'], [move['type']])
    
    hp = hp_stat_calc(targetDex['baseStats']['hp'], target['evs']['hp'], target['ivs']['hp'], level)
    
    dmg = (( (((2 * level) / 5) + 2) * power * atk/dfn ) / 50) * weatherEff * stab * typeEff
    
    return dmg / hp

In [11]:
def weather_effect(weather, moveType):
    if weather == RAINDANCE:
        if moveType == 'Water':
            return 1.5
        elif moveType == 'Fire':
            return 0.5
    elif weather == SUNNYDAY:
        if moveType == 'Fire':
            return 1.5
        elif moveType == 'Water':
            return 0.5
        
    return 1.0